# 2.5 Inter-spike interval モデル
これまで紹介したモデルでは、入力に対する膜電位などの時間変化に基づき発火が起こるかどうか、ということを考えてきました。この節では、発火が生じるまでの過程を考慮せず、発火の時間間隔(**inter-spike interval, ISI**)の統計による現象論的モデルを考えます(これを**Inter-spike interval (ISI)** モデルと呼びます)。ISIモデルは**点過程(point process)** という統計的モデルに基づいており、各モデルにはISIが従う分布の名称がついています。この節では、使用頻度の高い **ポアソン過程 (Poisson process) モデル**、ポアソン過程モデルにおいて不応期を考慮した **死時間付きポアソン過程 (Poisson process with dead time, PPD) モデル**、皮質の定常発火においてポアソン過程モデルよりも当てはまりがよいとされる **ガンマ過程 (Gamma process) モデル**について説明します。

なお、SNNにおいて、ISIモデルは主に画像入力の際に**連続値からスパイク列へのエンコード**に用いられます。

## 2.5.1 ポアソン過程モデル
### 点過程とポアソン過程
時間に応じて変化する確率変数のことを**確率過程(stochastic process)** といいます。さらに確率過程の中で、連続時間軸上において離散的に生起する点事象の系列を**点過程(point process)** といいます。スパイクは離散的に起こるので、点過程を用いてモデル化ができるという話です。

ポアソン過程 (Poisson process)は点過程の1つです。ポアソン過程モデルはスパイクの発生をポアソン過程でモデル化したもので、このモデルによって生じるスパイクをポアソンスパイク(Poisson spike)と呼びます。ポアソン過程では、時刻$t$までに起こった点の数$N(t)$はポアソン分布に従います。すなわち、点が起こる確率が強度$\lambda$のポアソン分布に従う場合, 時刻$t$までに事象が$n$回起こる確率は$P[N(t)=n]=\dfrac{(\lambda t)^{n}}{n !} e^{-\lambda t}$となります. 

ポアソン過程において点が起こる回数がポアソン分布に従うことは、ポアソン過程という名称の由来です。これを定義とする場合もあれば、次の4条件を満たす点過程をポアソン過程とするという定義もあります。

- 時刻0における初期の点の数は0 : $P[N(0)=0]=1$ 
- $[t, t+\Delta t)$に点が1つ生じる確率 : $P[N(t+\Delta t)-N(t)=1]=\lambda(t)\Delta t+o(\Delta t)$
- 微小時間$\Delta t$の間に点は2つ以上生じない : $P[N(t+\Delta t)-N(t)=2]=o(\Delta t)$
- 任意の時点$t_1 < t_2 < \cdots< t_n$に対して，増分 $N(t_2)-N(t_1), N(t_3)-N(t_2), \cdots, N(t_n)-N(t_{n−1})$は互いに独立である．

ただし, $o(\cdot)$はLandauの記号(Landauのsmall o)であり, $o(x)$は$x\to 0$のとき、$o(x)/x\to 0$となる微小な量を表します。ポアソン過程に従ってスパイクが生じるとする場合、条件2の強度関数$\lambda(t)$は**発火率**を意味します(また実装において有用)。条件3は不応期より小さいタイムステップにおいては、1つのタイムステップにおいて1つしかスパイクは生じないということを表します。条件4はスパイクは独立に発生する、ということを意味します。また、これらの条件から$N(t)$の分布は強度母数$\lambda(t)$のポアソン分布に従うことが示せます。

強度関数(点がスパイクの場合、発火率)が$\lambda(t)=\lambda$ (定数)となる場合は点の時間間隔(点がスパイクの場合、ISI)の確率変数$T$が強度母数$\lambda$の **指数分布**に従います。なお、指数分布の確率密度関数は確率変数を$T$とするとき、

$$
f(t;\lambda )=\left\{{\begin{array}{ll}\lambda e^{-\lambda t}&(t\geq 0)\\0&(t<0)\end{array}}\right.
$$

となります。このことは4条件とChapman-Kolmogorovの式により求められますが、ややこしいので, $P[N(t)=n]=\dfrac{(\lambda t)^{n}}{n !} e^{-\lambda t}$から導出できることを簡単に示します。指数分布の累積分布関数を$F(t; \lambda)$とすると、

$$
F(t; \lambda) = P(T<t)=1-P(T>t)=1-P(N(t)=0)=1-e^{-\lambda t}
$$

となります。よって

$$
f(t; \lambda)=\frac{dF(t; \lambda)}{dt}=\lambda e^{-\lambda t}
$$

となります。

### 定常ポアソン過程
ここからポアソン過程によるスパイクのシミュレーションを実装していきます。実装方法にはISIが指数分布に従うことを利用したものと、ポアソン過程の条件2を利用したものの2通りがあります。実装は後者が楽で計算量も少ないですが、後のガンマ過程のために前者の実装を先に行います。\par
ISIが指数分布に従うことを利用してポアソン過程モデルの実装を行います。SNNのシミュレーションにおいてスパイク列$S$はタイムステップごとに発火しているかの$\{0,1\}$配列で保持しておくと楽に扱うことができます\footnote{もちろん発火時刻で記録しておく方がメモリを節約できますが、計算量が多くなります。}。手順としては、まずISIを指数分布に従う乱数とします。次にISIを累積することで発火時刻を得ます。最後に発火時間を整数値に丸めてindexとすることで$\{0, 1\}$のスパイク列が得られます。ISIの取得には\texttt{numpy.random.exponential()}を用います。ややこしいのですが、この関数は引数として\texttt{scale}と\texttt{size}を受け取りますが、この\texttt{scale}は指数分布の確率密度関数を$f(t; \frac{1}{\beta}) = \frac{1}{\beta} e^{-t/\beta}$とした際の$\beta = 1/\lambda$です(この時、平均は$\beta$となります)。よって発火率を\texttt{fr}(1/s), 単位時間を\texttt{dt}(s)としたときのISIは
\texttt{isi = np.random.exponential(1/(fr*dt))}として得ることができます。

In [ ]:
np.random.seed(seed=0)
dt = 1e-3; T = 1; nt = round(T/dt) # シミュレーション時間
n_neurons = 10 # ニューロンの数
fr = 30 # ポアソンスパイクの発火率(Hz)
isi = np.random.exponential(1/(fr*dt),
                            size=(round(nt*1.5/fr), n_neurons))
spike_time = np.cumsum(isi, axis=0) # ISIを累積
spike_time[spike_time > nt - 1] = 0 # ntを超える場合を0に
spike_time = spike_time.astype(np.int32) # float to int
spikes = np.zeros((nt, n_neurons)) # スパイク記録変数
for i in range(n_neurons):    
    spikes[spike_time[:, i], i] = 1
spikes[0] = 0 # (spike_time=0)の発火を削除
print("Num. of spikes:", np.sum(spikes))
print("Firing rate:", np.sum(spikes)/(n_neurons*T))
# 描画
t = np.arange(nt)*dt
plt.figure(figsize=(5, 4))
for i in range(n_neurons):    
    plt.plot(t, spikes[:, i]*(i+1), 'ko', markersize=2)
plt.xlabel('Time (s)'); plt.ylabel('Neuron index') 
plt.xlim(0, T); plt.ylim(0.5, n_neurons+0.5)
plt.show()

結果は図のようになります。

次に2番目のポアソン過程モデルの実装を行います。こちらは$\lambda$を発火率とした場合, 区間$[t, t+\Delta t]$の間にポアソンスパイクが発生する確率は$\lambda \Delta t$となることを利用します。これはポアソン過程の条件ですが、ポアソン分布から導けることを簡単に示しておきます。事象が起こる確率が強度$\lambda$のポアソン分布に従う場合, 時刻$t$までに事象が$n$回起こる確率は$P[N(t)=n]=\dfrac{(\lambda t)^{n}}{n !} e^{-\lambda t}$となります. よって, 微小時間$\Delta t$において事象が$1$回起こる確率は
$$P[N(\Delta t)=1]=\dfrac{\lambda \Delta t}{1 !} e^{-\lambda \Delta t}\simeq \lambda \Delta t+o(\Delta t)$$
となります. ただし, $e^{-\lambda \Delta t}$についてはマクローリン展開による近似を行っています. このことから, 一様分布$U(0,1)$に従う乱数$\xi$を取得し, $\xi<\lambda dt$なら発火$(y=1)$, それ以外では$(y=0)$となるようにすればポアソンスパイクを実装できます.


In [ ]:
np.random.seed(seed=0)
dt = 1e-3; T = 1; nt = round(T/dt) # シミュレーション時間
n_neurons = 10 # ニューロンの数
fr = 30 # ポアソンスパイクの発火率(Hz)

# スパイク記録変数
spikes = np.where(np.random.rand(nt, n_neurons) < fr*dt, 1, 0)

print("Num. of spikes:", np.sum(spikes))
print("Firing rate:", np.sum(spikes)/(n_neurons*T))
# 描画
t = np.arange(nt)*dt
plt.figure(figsize=(5, 4))
for i in range(n_neurons):    
    plt.plot(t, spikes[:, i]*(i+1), 'ko', markersize=2)
plt.xlabel('Time (s)'); plt.ylabel('Neuron index') 
plt.xlim(0, T); plt.ylim(0.5, n_neurons+0.5)
plt.show()

結果は図\ref{fig:poisson_process}のようになります。
\caption{ポアソン過程モデルによる10個のニューロンのスパイクの1秒間のシミュレーション。(左)ISIの累積により発火時刻を求める手法。(右)$\Delta t$間の発火確率が$\lambda\Delta t$であることを利用する方法。結果は同一ですが、実行速度は右の方が速いです。}

なお、ここでは全時間における発火をまとめて計算していますが、タイムステップごとに発火の有無を計算することもできます。前者は発火情報を保持するためのメモリが必要ですが、計算時間は短くて済みます。後者はメモリの節約になりますが、計算時間は長くなります。そのため、これら2つの方法はメモリと計算時間のトレードオフとなります。また、他には発火情報を疎行列(sparse matrix)の形式で保持しておくとメモリの節約になると思われます。

### 非定常ポアソン過程
これまでの実装は発火率$\lambda$が一定であるとする、定常ポアソン過程 (homogeneous poisson process)でしたが、ここからは発火率$\lambda(t)$が時間変化するとする、非定常ポアソン過程 (inhomogeneous poisson process)を行います。とはいえ、定常ポアソン過程における発火率を、時間についての関数で置き換えるだけで実装できます。以下は$\lambda(t)=\sin^2(\alpha t)$(ただし$\alpha$は定数)とした場合の実装です

In [ ]:
np.random.seed(seed=0)
dt = 1e-3; T = 1; nt = round(T/dt) # シミュレーション時間
n_neuron = 10 # ニューロンの数
t = np.arange(nt)*dt

# ポアソンスパイクの発火率(Hz)
fr = np.expand_dims(30*np.sin(10*t)**2, 1)

# スパイク記録変数
spikes = np.where(np.random.rand(nt, n_neuron) < fr*dt, 1, 0)

print("Num. of spikes:", np.sum(spikes))
# 描画
plt.figure(figsize=(5, 4))
plt.subplot(2,1,1)
plt.plot(t, fr[:, 0], color="k")
plt.ylabel('Firing rate (Hz)') 
plt.xlim(0, T)

plt.subplot(2,1,2)
for i in range(n_neuron):    
    plt.plot(t, spikes[:, i]*(i+1), 'ko', markersize=2,
             rasterized=True)
plt.xlabel('Time (s)'); plt.ylabel('Neuron index') 
plt.xlim(0, T); plt.ylim(0.5, n_neuron+0.5)
plt.show()

結果は図\ref{fig:ipp}のようになります。
\begin{figure}[htbp]
 \begin{minipage}{0.5\hsize}
  \begin{center}
   \includegraphics[width=65mm]{figs/inhomogenous_poisson_process.pdf}
   \caption{非定常ポアソン過程による10個のニューロンのスパイクの1秒間のシミュレーション。(上)発火率$\lambda(t)$の時間変化, (下)ラスタープロット。}

## 2.5.2 死時間付きポアソン過程モデル (Poisson process with dead time, PPD)
ポアソン過程は簡易的で有用ですが、不応期を考慮していません。そのため、時には生理的範疇を超えたバースト発火が起こる場合もあります。そこで、ポアソン過程において不応期のようなイベントの生起が起こらない\textbf{死時間(dead time)}\footnote{例えば、ガイガー・カウンター(Geiger counter)などの放射線の検出器には放射線の到達を機器の物理的特性として検出できない時間(つまり死時間)があります。そのため放射線の到達数がポアソン分布に従うとした場合、放射線測定装置のモデルとしてPPDが用いられます。}
を考慮した\textbf{死時間付きポアソン過程 (Poisson process with dead time, PPD)}というモデルを導入します。\par
実装においてはLIFニューロンの時と同じような不応期の処理をします。つまり、現在が不応期かどうかを判断し、不応期なら発火を許可しないようにします

In [ ]:
np.random.seed(seed=0)
dt = 1e-3; T = 1; nt = round(T/dt) # シミュレーション時間
n_neurons = 10 # ニューロンの数
tref = 5e-3 # 不応期 (s)
fr = 30 # ポアソンスパイクの発火率(Hz)
spikes = np.zeros((nt, n_neurons)) #スパイク記録変数
tlast = np.zeros(n_neurons) # 発火時刻の記録変数
for i in range(nt):
    s = np.where(np.random.rand(n_neurons) < fr*dt, 1, 0)
    spikes[i] = ((dt*i) > (tlast + tref))*s
    tlast = tlast*(1-s) + dt*i*s # 発火時刻の更新

print("Num. of spikes:", np.sum(spikes))
print("Firing rate:", np.sum(spikes)/(n_neurons*T))
# 描画
t = np.arange(nt)*dt
plt.figure(figsize=(5, 4))
for i in range(n_neurons):    
    plt.plot(t, spikes[:, i]*(i+1), 'ko', markersize=2)
plt.xlabel('Time (s)'); plt.ylabel('Neuron index') 
plt.xlim(0, T); plt.ylim(0.5, n_neurons+0.5)
plt.show()

## 2.5.3 ガンマ過程モデル
ガンマ過程(gamma process)は点の時間間隔がガンマ分布に従うとするモデルです。ガンマ過程はポアソン過程よりも皮質における定常発火への当てはまりが良いとされています(Shinomoto, et al., 2003; Maimon,\& Assad,2009).

時間間隔の確率変数を$T$とした場合、ガンマ分布の確率密度関数は
\begin{equation}
f(t;k,\theta) =  t^{k-1}\frac{e^{-t/\theta}}{\theta^k\Gamma(k)}
\end{equation}
と表されます。ただし、$t > 0$であり、 2つの母数は$k, \theta > 0$です。また、$\Gamma (\cdot)$はガンマ関数であり、
\begin{equation}
\Gamma (k)=\int _{0}^{\infty }x^{k-1}e^{-x}\,dx
\end{equation}
と定義されます。ガンマ分布の平均は$k\theta$ですが、発火率はISIの平均の逆数なので、$\lambda=1/k\theta$となります。また、$k=1$のとき、ガンマ分布は指数分布となります。さらに$k$が正整数のとき、ガンマ分布はアーラン分布となります。\par
ガンマ過程モデルの実装はポアソン過程モデルのISIを累積する手法と同様に書くことができ、\texttt{numpy.random.exponential()}を\texttt{numpy.random.gamma()}に置き換えるだけです(もちろん多少の修正は必要とします)。また、ガンマ分布の描画にガンマ関数を用いるので、\texttt{import scipy.special as sps}によりガンマ関数の関数\texttt{sps.gamma()}を使用できるようにします

In [ ]:
import scipy.special as sps
np.random.seed(seed=0)
dt = 1e-3; T = 1; nt = round(T/dt) # シミュレーション時間
n_neurons = 10 # ニューロンの数
fr = 30 # ガンマスパイクの発火率(Hz)
k = 12 # k=1のときはポアソン過程に一致
theta = 1/(k*(fr*dt)) # fr = 1/(k*theta)
isi = np.random.gamma(shape=k, scale=theta,
                      size=(round(nt*1.5/fr), n_neurons))
spike_time = np.cumsum(isi, axis=0) # ISIを累積
spike_time[spike_time > nt - 1] = -1 # ntを超える場合を0に
spike_time = spike_time.astype(np.int32) # float to int
spikes = np.zeros((nt, n_neurons)) # スパイク記録変数
for i in range(n_neurons):    
    spikes[spike_time[:, i], i] = 1
spikes[0] = 0 # (spike_time=0)の発火を削除
print("Num. of spikes:", np.sum(spikes))
print("Firing rate:", np.sum(spikes)/(n_neurons*T))

# 描画
plt.figure(figsize=(5, 5))
t = np.arange(nt)*dt
plt.subplot(2,1,1)
count, bins, ignored = plt.hist(isi.flatten(),
                                50, density=True,
                                color="gray", alpha=0.5)
y = bins**(k-1)*(np.exp(-bins/theta) / (sps.gamma(k)*theta**k))
plt.plot(bins, y, linewidth=2, color="k")
plt.title('k='+str(k)) 
plt.xlabel('ISI (ms)') 
plt.ylabel('Probability density') 

plt.subplot(2,1,2)
for i in range(n_neurons):    
    plt.plot(t, spikes[:, i]*(i+1), 'ko', markersize=2)
plt.xlabel('Time (s)'); plt.ylabel('Neuron index') 
plt.xlim(0, T); plt.ylim(0.5, n_neurons+0.5)
plt.show()

ガンマ過程モデルによる10個のニューロンのスパイクの1秒間のシミュレーション。上段はISIの分布(横軸の単位がmsであることに注意)。下段はラスタープロット。左の$k=1$の場合はポアソン過程モデルと同じです。右は$k=12$とした場合です。}
\label{fig:gamma_process}
\end{figure}
なお、前述したようにガンマ過程モデルの方がポアソン過程モデルよりも皮質ニューロンのモデルとしては優れていますが、入力画像のエンコーディングをガンマ過程モデルにすることでSNNの認識精度が向上するかどうかはまだ十分に研究されていません。また、(Deger, et al., 2012)ではPPDやガンマ過程の重ね合わせによるスパイク列を生成するアルゴリズムを考案しています。